In [26]:
import pandas as pd
import requests
import json
import html
import datetime
import csv
import re
from datetime import date

In [2]:
companies_df = pd.read_csv('../lever_companies.csv')
companies_df


,Company,Token
0,Anduril Industries,anduril
1,Angle Health,AngleHealth
2,Anyscale,anyscale
3,Ascend.io,ascend.io
4,Asics,?asics?
...,...,...
57,Up&Up,Up&Up
58,Veeva,veeva
59,Xage Security,xage-security
60,YugabyteDB,yugabyte


In [3]:
companies_dict = {}
for _ , row in companies_df.iterrows():
  companies_dict[row['Token']] = row['Company']


companies_dict


{'anduril': 'Anduril Industries',
 'AngleHealth': 'Angle Health',
 'anyscale': 'Anyscale',
 'ascend.io': 'Ascend.io',
 '?asics?': 'Asics',
 'atlassian': 'Atlassian',
 'aurorasolar': 'Aurora Solar',
 'automox': 'Automox',
 'axoni': 'Axoni',
 '?belvedereTrading?': 'Belvedere Trading',
 'bluebottlecoffee': 'Blue Bottle Coffee',
 'bodyport': 'Bodyport',
 'boomandbucket': 'Boom & Bucket',
 '?butcherbox?': 'Butcher Box',
 '?captiv8?': 'Captiv8',
 'cellares': 'Cellares',
 'cohere': 'Cohere',
 'confluent': 'Confluent',
 'credinvestments': 'CRED Investments',
 'eargo': 'Eargo',
 ' facta': 'Facta',
 'goforward': 'Forward',
 'front': 'Front',
 'getlabs': 'Getlabs',
 'gopuff': 'Gopuff',
 '?greenthumbindustries?': 'Green Thumb industr.',
 'hihello': 'HiHello',
 'hive': 'Hive',
 'learnwithjuni': 'Juni Learning',
 'leolabs-2': 'LeoLabs',
 'logrocket': 'LogRocket',
 'MachinaLabs': 'Machina Labs',
 'nava': 'Nava',
 'neeva': 'Neeva',
 'netflix': 'Netflix',
 'nexient': 'Nexient',
 'observeinc': 'Observe'

In [4]:
tokens = set(companies_df.Token)
companies_clean = companies_dict
companies_bad = {}

In [34]:
with open('../lever_companies.csv', 'r') as data:
    companies = {row[0]: row[1] for row in csv.reader(data)}
            
companies_bad_lever = {}
tokens = set(companies.keys())
companies_clean = companies

for token in tokens:
    if not requests.get(f'https://jobs.lever.co/v0/postings/{token}?mode=json'):
        companies_bad_lever[token] = companies.get(token)
        companies_clean.pop(token)

if companies_bad_lever:
    with open(f"../logs/companies_bad_lever_{date.today()}.json", "w") as out:
        json.dump(companies_bad_lever, out)

tokens

{'?asics?',
 '?belvedereTrading?',
 '?butcherbox?',
 '?captiv8?',
 '?greenthumbindustries?',
 '?perpay?',
 'AngleHealth',
 'MachinaLabs',
 'OpenSea',
 'Pomelo',
 'Sherpa',
 'Up&Up',
 'anduril',
 'anyscale',
 'ascend.io',
 'atlassian',
 'aurorasolar',
 'automox',
 'axoni',
 'bluebottlecoffee',
 'bodyport',
 'boomandbucket',
 'cellares',
 'cohere',
 'confluent',
 'credinvestments',
 'eargo',
 'facta',
 'front',
 'getlabs',
 'goforward',
 'gopuff',
 'hihello',
 'hive',
 'learnwithjuni',
 'leolabs-2',
 'logrocket',
 'nava',
 'neeva',
 'netflix',
 'nexient',
 'observeinc',
 'openbb',
 'orum',
 'ottertune',
 'picktrace',
 'plaid',
 'roboflow',
 'secureframe',
 'sporty',
 'spotify ',
 'starburst',
 'sure',
 'talyn',
 'tegus',
 'terminus',
 'torch',
 'translifeline',
 'veeva',
 'xage-security',
 'yugabyte',
 'zoox'}

In [37]:
len(companies_clean)

50

In [6]:
for token in tokens:
    if not requests.get(f'https://jobs.lever.co/v0/postings/{token}?mode=json'):
      print(f'API call for {token} failed')
      companies_bad[token] = companies_dict.get(token)
      companies_clean.pop(token)

API call for ?captiv8? failed
API call for spotify  failed
API call for ?greenthumbindustries? failed
API call for starburst failed
API call for  translifeline failed
API call for boomandbucket failed
API call for ?perpay? failed
API call for ?butcherbox? failed
API call for  facta failed
API call for  sure failed
API call for ?belvedereTrading? failed
API call for ?asics? failed
API call for  Up&Up failed
API call for front failed


In [7]:
companies_bad

{'?captiv8?': 'Captiv8',
 'spotify ': 'Spotify',
 '?greenthumbindustries?': 'Green Thumb industr.',
 'starburst': 'Starburst',
 ' translifeline': 'TransLifeline',
 'boomandbucket': 'Boom & Bucket',
 '?perpay?': 'Perpay',
 '?butcherbox?': 'Butcher Box',
 ' facta': 'Facta',
 ' sure': 'Sure',
 '?belvedereTrading?': 'Belvedere Trading',
 '?asics?': 'Asics',
 ' Up&Up': 'Up&Up',
 'front': 'Front'}

In [8]:
roles = {'developer','engineer', 'data', 'engineering', 'frontend', 'software', 'apprentice', 'analyst' , 'quality', 'apprenticeship', 'front-end', 'backend', 'back-end', 'jr.', 'jr' }
# Includes 'software' for titles that are just 'software enginer', etc.
levels = {'junior', 'entry-level', 'grad', 'graduate', 'data', 'apprentice', 'apprenticeship', 'software', 'qa', 'quality', 'test', 'entry', 'intern', 'i', '1', 'associate', 'co-op'}
# Optional, but helps exclude higher level postiions
exclude = {'senior', 'principal' , 'sr.', 'sr' 'ii', 'iii', 'director' }



In [9]:
res=requests.get(f'https://jobs.lever.co/v0/postings/tegus?mode=json')
jobs=json.loads(res.text)
job = jobs[0]

job.keys()

dict_keys(['additionalPlain', 'additional', 'categories', 'createdAt', 'descriptionPlain', 'description', 'id', 'lists', 'text', 'country', 'workplaceType', 'hostedUrl', 'applyUrl'])

In [10]:
job['categories']

{'commitment': 'Full-Time',
 'department': 'Operations',
 'location': 'Chicago, IL',
 'team': 'Operations'}

In [11]:
print(datetime.datetime.fromtimestamp(job.get('createdAt')/1000.).isoformat())


2020-12-10T18:18:24.613000


In [21]:
results = []

for token in tokens:
  res=requests.get(f'https://jobs.lever.co/v0/postings/{token}?mode=json')
  if res:
    jobs=json.loads(res.text)

  company= companies_clean.get(token)

  for job in jobs:
    title = set(job.get('text').lower().split())

    if title.intersection(roles) and title.intersection(levels) and not title.intersection(exclude):
      job_info = {'title': job.get('text'), 'company' : company, 'description': job.get('descriptionPlain'),
    'link' : job.get('applyUrl'), 'remote' : True if job.get('workplaceType').lower() == 'remote' else False, 'location' : job.get('categories').get('location') ,  'date_posted': datetime.datetime.fromtimestamp(int(str(job.get('createdAt'))[:-3])).isoformat()} 
      
      results.append(job_info)



In [19]:
str(1679440926995)[:-3]


'1679440926'

In [22]:
df = pd.DataFrame()
df = df.from_records(results)
df

,title,company,description,link,remote,location,date_posted
0,Cloud Optimization Data Analyst,None,"Who are we?\nWe’re a small, diverse team worki...",https://jobs.lever.co/cohere/91abe8c4-da02-450...,False,"Toronto, Palo Alto, San Francisco",2023-03-21T19:22:06
1,Data Quality Specialist (Part-Time),None,"Who are we?\nWe’re a small, diverse team worki...",https://jobs.lever.co/cohere/486e96ac-ac4f-40c...,False,Toronto,2023-01-30T13:01:09
2,Data Quality Specialist (Part-Time),None,"Who are we?\nWe’re a small, diverse team worki...",https://jobs.lever.co/cohere/e461b759-d102-44f...,False,London,2023-03-16T10:17:00
3,"Member of Technical Staff, Data Acquisition",None,"Who are we?\nWe’re a small, diverse team worki...",https://jobs.lever.co/cohere/c282d795-efe1-481...,True,Remote,2023-03-30T16:20:56
4,"Member of Technical Staff, Data Quality",None,"Who are we?\nWe’re a small, diverse team worki...",https://jobs.lever.co/cohere/dbb767b7-3a95-4af...,True,Remote,2023-04-25T14:33:08
...,...,...,...,...,...,...,...
497,Cloud Optimization Data Analyst,Cohere,"Who are we?\nWe’re a small, diverse team worki...",https://jobs.lever.co/cohere/91abe8c4-da02-450...,False,"Toronto, Palo Alto, San Francisco",2023-03-21T19:22:06
498,Data Quality Specialist (Part-Time),Cohere,"Who are we?\nWe’re a small, diverse team worki...",https://jobs.lever.co/cohere/486e96ac-ac4f-40c...,False,Toronto,2023-01-30T13:01:09
499,Data Quality Specialist (Part-Time),Cohere,"Who are we?\nWe’re a small, diverse team worki...",https://jobs.lever.co/cohere/e461b759-d102-44f...,False,London,2023-03-16T10:17:00
500,"Member of Technical Staff, Data Acquisition",Cohere,"Who are we?\nWe’re a small, diverse team worki...",https://jobs.lever.co/cohere/c282d795-efe1-481...,True,Remote,2023-03-30T16:20:56


In [14]:
df.sort_values(by='date_posted', ascending=False)

,title,company,description,link,remote,location,date_posted
258,Data Steward Supervisor - French Speaker,Veeva,Veeva is a mission-driven organization that as...,https://jobs.lever.co/veeva/93ec49c7-518d-4d91...,True,Hungary - Budapest,2023-05-12T10:26:50.847000
207,Analyst - Computer Software Validation,Veeva,Veeva is a mission-driven organization that as...,https://jobs.lever.co/veeva/75768d72-de7a-41a5...,True,Croatia - Zagreb,2023-05-12T09:15:43.565000
392,Manufacturing Engineer Co-op Summer/Winter (An...,Anduril Industries,Anduril Industries is a defense technology com...,https://jobs.lever.co/anduril/0fe84ca6-54e3-46...,False,"Quincy, MA",2023-05-11T18:54:11.710000
272,Engagement Manager - Business Consulting - Qua...,Veeva,Veeva is a mission-driven organization that as...,https://jobs.lever.co/veeva/5fd9f8c8-f960-4ae2...,True,Germany - Berlin,2023-05-11T12:12:41.569000
271,Engagement Manager - Business Consulting - Qua...,Veeva,Veeva is a mission-driven organization that as...,https://jobs.lever.co/veeva/346b460f-2a75-481f...,True,Spain - Barcelona,2023-05-11T12:11:29.055000
...,...,...,...,...,...,...,...
388,Frontend Software Engineer,Anduril Industries,Anduril Industries is a defense technology com...,https://jobs.lever.co/anduril/a79b3c52-aaf8-47...,False,"Costa Mesa, CA",2018-07-17T18:19:39.522000
174,Technical Program Manager - Software,Zoox,Zoox has embarked on a highly ambitious journe...,https://jobs.lever.co/zoox/b5b5bb0c-1f95-446e-...,False,"Foster City, CA",2016-05-17T13:37:06.372000
105,Senior/Staff Software Engineer - Machine Learning,Zoox,"At Zoox, you will collaborate with a team of w...",https://jobs.lever.co/zoox/81f7fbe7-4fd3-4814-...,False,"Foster City, CA",2016-02-11T20:45:48.453000
135,Software Development Engineer in Test,Zoox,"Autonomous vehicles have some of the largest, ...",https://jobs.lever.co/zoox/206cbbe7-f2ef-40f1-...,False,"Foster City, CA",2016-02-11T17:06:53.811000
